In [1]:
from minichat.common import get_base_dir
import os
import fcntl
import urllib.request
import tempfile, zipfile
import shutil
import hashlib
import yaml
import csv
import json

In [2]:
def download_file_with_lock(url, filename, postprocess_fn=None):
    """
    Downloads a file from a URL to a local path in the base directory.
    Uses a lock file to prevent concurrent downloads among multiple ranks.
    """
    base_dir = get_base_dir()
    file_path = os.path.join(base_dir, filename)
    lock_path = file_path + ".lock"

    if os.path.exists(file_path):
        return file_path

    with open(lock_path, 'w', encoding='utf-8') as lock_file:

        # Only a single rank can acquire this lock
        # All other ranks block until it is released
        fcntl.flock(lock_file.fileno(), fcntl.LOCK_EX)

        # Recheck after acquiring lock (another process may have downloaded it)
        if os.path.exists(file_path):
            return file_path

        # Download the content as bytes
        print(f"Downloading {url}...")
        with urllib.request.urlopen(url) as response:
            content = response.read() # bytes

        # Write to local file
        with open(file_path, 'wb') as f:
            f.write(content)
        print(f"Downloaded to {file_path}")

        # Run the postprocess function if provided
        if postprocess_fn is not None:
            postprocess_fn(file_path)

    # Clean up the lock file after the lock is released
    try:
        os.remove(lock_path)
    except OSError:
        pass  # Ignore if already removed by another process

def place_eval_bundle(file_path):
    # here file_path is the path to the eval_bundle.zip file
    # we need to unzip it and place it in the base directory
    base_dir = get_base_dir()
    eval_bundle_dir = os.path.join(base_dir, "eval_bundle")
    with tempfile.TemporaryDirectory() as tmpdir:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(tmpdir)
        extracted_bundle_dir = os.path.join(tmpdir, "eval_bundle")
        shutil.move(extracted_bundle_dir, eval_bundle_dir)
    print(f"Placed eval_bundle directory at {eval_bundle_dir}")

In [ ]:
EVAL_BUNDLE_URL = "https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip"

In [5]:
download_file_with_lock(EVAL_BUNDLE_URL, "eval_bundle.zip", postprocess_fn=place_eval_bundle)

NameError: name 'download_file_with_lock' is not defined

In [5]:
base_dir = get_base_dir()
eval_bundle_dir = os.path.join(base_dir, "eval_bundle")
config_path = os.path.join(eval_bundle_dir, "core.yaml")
data_base_path = os.path.join(eval_bundle_dir, "eval_data")
eval_meta_data = os.path.join(eval_bundle_dir, "eval_meta_data.csv")

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

In [6]:
tasks = config['icl_tasks']

In [7]:
random_baselines = {}
with open(eval_meta_data, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        task_name = row['Eval Task']
        random_baseline = row['Random baseline']
        random_baselines[task_name] = float(random_baseline)

In [8]:
task = tasks[0]  # Example: get the first task

label = task['label']
task_meta = {
    'task_type': task['icl_task_type'],
    'dataset_uri': task['dataset_uri'],
    'num_fewshot': task['num_fewshot'][0],
    'continuation_delimiter': task.get('continuation_delimiter', ' ')
}
        

In [9]:
label, task_meta

('hellaswag_zeroshot',
 {'task_type': 'multiple_choice',
  'dataset_uri': 'language_understanding/hellaswag.jsonl',
  'num_fewshot': 0,
  'continuation_delimiter': ' '})

In [10]:
data_path = os.path.join(data_base_path, task_meta['dataset_uri'])
with open(data_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line.strip()) for line in f.readlines()]

In [11]:
import random
import torch

max_per_task = 10

shuffle_rng = random.Random(1337)
shuffle_rng.shuffle(data)
data = data[:max_per_task]


correct = torch.zeros(len(data))
for i in range(len(data)):
    example = data[i]
    task_type = task_meta['task_type']
    num_fewshot = task_meta['num_fewshot']
    continuation_delimiter = task_meta['continuation_delimiter']

    fewshot_examples = []
    if num_fewshot > 0:
        rng = random.Random(42 + i)  # Seed with a combination of a constant and the example index
        available_indices = [idx for idx in range(len(data)) if idx != i]
        fewshot_indices = rng.sample(available_indices, num_fewshot)
        fewshot_examples = [data[idx] for idx in fewshot_indices]         
    


In [12]:
from jinja2 import Template

template_str = """
{%- for ex in fewshot_examples %}
{{ex.query}}{{ continuation_delimiter }}{{ ex.choices[ex.gold] }}

{%- endfor %}
{{item.query}}{{ continuation_delimiter }}{{choice}}
""".strip() 

template = Template(template_str)

In [13]:
prompts = [template.render(
    fewshot_examples=fewshot_examples,
    item=example,
    choice=choice,
    continuation_delimiter=continuation_delimiter)
    for choice in example['choices']]

In [14]:
from minichat.tokenizer import get_tokenizer
tokenizer = get_tokenizer()

In [15]:
def find_common_length(prompts, direction='left'):
    """
    Finds the common length of tokenized prompts.
    direction: 'left' or 'right' - indicates which side to consider for commonality
    """
    tokenized_prompts = [tokenizer(prompt) for prompt in prompts]
    if direction == 'left':
        min_length = min(len(tokens) for tokens in tokenized_prompts)
        common_length = 0
        for i in range(min_length):
            current_token = tokenized_prompts[0][i]
            if all(tokens[i] == current_token for tokens in tokenized_prompts):
                common_length += 1
            else:
                break
        return common_length
    elif direction == 'right':
        min_length = min(len(tokens) for tokens in tokenized_prompts)
        common_length = 0
        for i in range(1, min_length + 1):
            current_token = tokenized_prompts[0][-i]
            if all(tokens[-i] == current_token for tokens in tokenized_prompts):
                common_length += 1
            else:
                break
        return common_length
    else:
        raise ValueError("Direction must be 'left' or 'right'")

In [28]:
tokens = tokenizer(prompts, prepand=tokenizer.get_bos_token_id())
start_indices = [find_common_length(prompts, direction='left')]* len(prompts)
end_indices = [len(p) for p in prompts]

In [29]:
start_indices

[59, 59, 59, 59]

In [30]:
def stack_sequences(tokens, pad_token_id):
    bsz, seq_len = len(tokens), max(len(t) for t in tokens)
    input_ids = torch.full((bsz, seq_len), pad_token_id, dtype=torch.long)
    for i, x in enumerate(tokens):
        input_ids[i, :len(x)] = torch.tensor(x, dtype=torch.long)
    return input_ids

In [35]:
def forward_model(model, input_ids):
    batch_size, seq_len = input_ids.shape
    outputs = model(input_ids)
    target_ids = torch.roll(input_ids, shifts=-1, dims=1)
    losses = torch.nn.functional.cross_entropy(
        outputs.view(batch_size*seq_len, -1),
        target_ids.view(batch_size*seq_len),
        reduction='none'
    ).view(batch_size, seq_len)
    losses[:, -1] = float('nan')  # Ignore loss for the last token
    predictions = torch.argmax(outputs, dim=-1)
    return losses, predictions

In [ ]:
max_token_len = 1024
new_tokens, new_start_ids, new_end_ids = [], [], []

for t, s, e in zip(tokens, start_indices, end_indices):
    if len(t) > max_token_len:
        num_to_crop = len(t) - max_token_len
        new_tokens.append(t[-max_token_len:])
        new_start_ids.append(s-num_to_crop)
        new_end_ids.append(e-num_to_crop)
    else:
        new_tokens.append(t)
        new_start_ids.append(s)
        new_end_ids.append(e)
    
tokens, start_ids, end_ids = new_tokens, new_start_ids, new_end_ids

pad_token_id = tokenizer.get_bos_token_id()
input_ids = stack_sequences(tokens, pad_token_id)

from minichat.gpt import GPT, GPTConfig


config = GPTConfig(
    sequence_len = 512,
    n_layers = 12,
    vocab_size = 50257,
    emb_dim = 128,
    n_heads = 8
)

model = GPT(config)
losses, predictions = forward_model(model, input_ids)

mean_losses = [losses[i, start_ids[i]:end_ids[i]-1].mean().item() for i in range(len(tokens))]  # exclude last token
pred_idx = mean_losses.index(min(mean_losses))

is_correct = (pred_idx == example['gold'])

print(f"Example {i}: Prediction index = {pred_idx}, Gold index = {example['gold']}, Correct = {is_correct}")
   

Example 9: Prediction index = 0, Gold index = 1, Correct = False


In [1]:
from minichat.common import download_file_with_lock, place_eval_bundle

EVAL_BUNDLE_URL = "https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip"

download_file_with_lock(
    EVAL_BUNDLE_URL,
    "eval_bundle.zip",
    postprocess_fn=place_eval_bundle
)

Downloaded to /home/shadeform/minichat/minichat/eval_bundle.zip
Placed eval_bundle directory at /home/shadeform/minichat/minichat/eval_bundle


In [2]:
from minichat.gpt import GPT, GPTConfig


config = GPTConfig(
    sequence_len = 512,
    n_layers = 12,
    vocab_size = 50257,
    emb_dim = 128,
    n_heads = 8
)

model = GPT(config)

In [3]:
from minichat.tokenizer import get_tokenizer
tokenizer = get_tokenizer()

In [7]:
import os

from minichat.common import get_base_dir

eval_bundle_dir = os.path.join(get_base_dir(), "eval_bundle")

config_path = os.path.join(eval_bundle_dir, "core.yaml")
data_base_path = os.path.join(eval_bundle_dir, "eval_data")
eval_meta_data = os.path.join(eval_bundle_dir, "eval_meta_data.csv")

In [13]:
import csv
import yaml

random_baselines = {}
with open(eval_meta_data, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        task_name = row['Eval Task']
        random_baseline = row['Random baseline']
        random_baselines[task_name] = float(random_baseline)


In [14]:
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)
tasks = config['icl_tasks']

In [11]:
random_baselines

{'mmlu_zeroshot': 25.0,
 'hellaswag_zeroshot': 25.0,
 'jeopardy': 0.0,
 'triviaqa_sm_sub': 0.0,
 'gsm8k_cot': 0.0,
 'agi_eval_sat_math_cot': 0.0,
 'aqua_cot': 0.0,
 'svamp_cot': 0.0,
 'bigbench_qa_wikidata': 0.0,
 'arc_easy': 25.0,
 'arc_challenge': 25.0,
 'mmlu_fewshot': 25.0,
 'bigbench_misconceptions': 50.0,
 'copa': 50.0,
 'siqa': 33.3,
 'commonsense_qa': 20.0,
 'piqa': 50.0,
 'openbook_qa': 25.0,
 'bigbench_novel_concepts': 20.0,
 'bigbench_strange_stories': 50.0,
 'bigbench_strategy_qa': 50.0,
 'lambada_openai': 0.0,
 'hellaswag': 25.0,
 'winograd': 50.0,
 'winogrande': 50.0,
 'bigbench_conlang_translation': 0.0,
 'bigbench_language_identification': 9.1,
 'bigbench_conceptual_combinations': 25.0,
 'bigbench_elementary_math_qa': 20.0,
 'bigbench_dyck_languages': 0.0,
 'agi_eval_lsat_ar': 20.0,
 'bigbench_cs_algorithms': 0.0,
 'bigbench_logical_deduction': 20.0,
 'bigbench_operators': 0.0,
 'bigbench_repeat_copy_logic': 0.0,
 'simple_arithmetic_nospaces': 0.0,
 'simple_arithmetic_w

In [15]:
tasks

[{'label': 'hellaswag_zeroshot',
  'dataset_uri': 'language_understanding/hellaswag.jsonl',
  'num_fewshot': [0],
  'icl_task_type': 'multiple_choice'},
 {'label': 'jeopardy',
  'dataset_uri': 'world_knowledge/jeopardy_all.jsonl',
  'num_fewshot': [10],
  'icl_task_type': 'language_modeling',
  'continuation_delimiter': '\nAnswer: ',
  'has_categories': True},
 {'label': 'bigbench_qa_wikidata',
  'dataset_uri': 'world_knowledge/bigbench_qa_wikidata.jsonl',
  'num_fewshot': [10],
  'icl_task_type': 'language_modeling'},
 {'label': 'arc_easy',
  'dataset_uri': 'world_knowledge/arc_easy.jsonl',
  'num_fewshot': [10],
  'icl_task_type': 'multiple_choice',
  'continuation_delimiter': '\nAnswer: '},
 {'label': 'arc_challenge',
  'dataset_uri': 'world_knowledge/arc_challenge.jsonl',
  'num_fewshot': [10],
  'icl_task_type': 'multiple_choice',
  'continuation_delimiter': '\nAnswer: '},
 {'label': 'copa',
  'dataset_uri': 'commonsense_reasoning/copa.jsonl',
  'num_fewshot': [0],
  'icl_task_typ

In [18]:
label = tasks[1]['label']
task_meta = {
    'task_type': tasks[1]['icl_task_type'],
    'dataset_uri': tasks[1]['dataset_uri'],
    'num_fewshot': tasks[1]['num_fewshot'][0],
    'continuation_delimiter': tasks[1].get('continuation_delimiter', ' ')
}

In [19]:
task_meta

{'task_type': 'language_modeling',
 'dataset_uri': 'world_knowledge/jeopardy_all.jsonl',
 'num_fewshot': 10,
 'continuation_delimiter': '\nAnswer: '}

In [25]:
import json
from minichat.core_eval import evaluate_task

data_path = os.path.join(data_base_path, task_meta['dataset_uri'])

with open(data_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line.strip()) for line in f.readlines()]

is_correct = evaluate_task(data, task_meta, model, tokenizer, device='cuda')

TypeError: evaluate_example() missing 1 required positional argument: 'device'

In [ ]:
from minichat.common import download_file_with_lock, place_eval_bundle
from minichat.gpt import GPT, GPTConfig
from minichat.tokenizer import get_tokenizer
import os
import csv
import yaml
from minichat.common import get_base_dir
import json
from minichat.core_eval import evaluate_task


EVAL_BUNDLE_URL = "https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip"

download_file_with_lock(
    EVAL_BUNDLE_URL,
    "eval_bundle.zip",
    postprocess_fn=place_eval_bundle
)

config = GPTConfig(
    sequence_len = 512,
    n_layers = 12,
    vocab_size = 50257,
    emb_dim = 128,
    n_heads = 8
)

model = GPT(config)

model.to('cuda')

tokenizer = get_tokenizer()

eval_bundle_dir = os.path.join(get_base_dir(), "eval_bundle")

config_path = os.path.join(eval_bundle_dir, "core.yaml")
data_base_path = os.path.join(eval_bundle_dir, "eval_data")
eval_meta_data = os.path.join(eval_bundle_dir, "eval_meta_data.csv")

random_baselines = {}
with open(eval_meta_data, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        task_name = row['Eval Task']
        random_baseline = row['Random baseline']
        random_baselines[task_name] = float(random_baseline)

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)
tasks = config['icl_tasks']

task_meta = {
    'task_type': tasks[1]['icl_task_type'],
    'dataset_uri': tasks[1]['dataset_uri'],
    'num_fewshot': tasks[1]['num_fewshot'][0],
    'continuation_delimiter': tasks[1].get('continuation_delimiter', ' ')
}

data_path = os.path.join(data_base_path, task_meta['dataset_uri'])

with open(data_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line.strip()) for line in f.readlines()]

is_correct = evaluate_task(data, task_meta, model, tokenizer, device='cuda')

AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [1]:
from minichat.common import download_file_with_lock, place_eval_bundle
from minichat.gpt import GPT, GPTConfig
from minichat.tokenizer import get_tokenizer
import os
import csv
import yaml
from minichat.common import get_base_dir
import json
from minichat.core_eval import evaluate_task


EVAL_BUNDLE_URL = "https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip"

download_file_with_lock(
    EVAL_BUNDLE_URL,
    "eval_bundle.zip",
    postprocess_fn=place_eval_bundle
)

config = GPTConfig(
    sequence_len = 512,
    n_layers = 12,
    vocab_size = 50257,
    emb_dim = 128,
    n_heads = 8
)

model = GPT(config)

model.to('cuda')

tokenizer = get_tokenizer()

eval_bundle_dir = os.path.join(get_base_dir(), "eval_bundle")

config_path = os.path.join(eval_bundle_dir, "core.yaml")
data_base_path = os.path.join(eval_bundle_dir, "eval_data")
eval_meta_data = os.path.join(eval_bundle_dir, "eval_meta_data.csv")

random_baselines = {}
with open(eval_meta_data, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        task_name = row['Eval Task']
        random_baseline = row['Random baseline']
        random_baselines[task_name] = float(random_baseline)

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)
tasks = config['icl_tasks']

task_meta = {
    'task_type': tasks[1]['icl_task_type'],
    'dataset_uri': tasks[1]['dataset_uri'],
    'num_fewshot': tasks[1]['num_fewshot'][0],
    'continuation_delimiter': tasks[1].get('continuation_delimiter', ' ')
}

data_path = os.path.join(data_base_path, task_meta['dataset_uri'])

with open(data_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line.strip()) for line in f.readlines()]

is_correct = evaluate_task(data, task_meta, model, tokenizer, device='cuda')

/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [148,0,0], thread: [0,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [148,0,0], thread: [1,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [148,0,0], thread: [2,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [148,0,0], thread: [3,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [148,0,0], thread: [4,0,0] Asser

AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [2]:
# Debug: Check vocabulary size mismatch
print(f"Tokenizer vocab size: {tokenizer.get_vocab_size()}")
print(f"Model vocab size: 50257")

# Check if any token IDs are out of bounds
sample_text = "Hello world"
token_ids = tokenizer(sample_text)
print(f"Sample token IDs: {token_ids}")
print(f"Max token ID: {max(token_ids)}")
print(f"Min token ID: {min(token_ids)}")

if max(token_ids) >= 50257:
    print("ERROR: Token IDs exceed model vocabulary size!")
else:
    print("Token IDs are within bounds")

Tokenizer vocab size: 65536
Model vocab size: 50257
Sample token IDs: [28475, 891]
Max token ID: 28475
Min token ID: 891
Token IDs are within bounds


In [3]:
# FIX: Use correct vocabulary size from tokenizer
tokenizer = get_tokenizer()

config = GPTConfig(
    sequence_len = 512,
    n_layers = 12,
    vocab_size = tokenizer.get_vocab_size(),  # Use actual tokenizer vocab size
    emb_dim = 128,
    n_heads = 8
)

model = GPT(config)
model.to('cuda')
model.eval()  # Set to evaluation mode

print(f"Model created with vocab_size: {config.vocab_size}")
print(f"Model is on device: {next(model.parameters()).device}")
print(f"Model is in eval mode: {not model.training}")

AcceleratorError: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
